# TP MNIST
Dans ce TP, vous allez utiliser la base de données de reconnaissance de chiffres manuscrits MNIST pour entraîner un réseau de neurones.

## Configuration
### Si vous utilisez un ordinateur de l'Enseirb:
#### 1) Lancer une session linux (et non pas windows)
#### 2) Aller dans "Applications", puis "Autre", puis "conda_pytorch" (un terminal devrait s'ouvrir)
#### 3) Dans ce terminal, taper la commande suivante pour lancer Spyder :  
`spyder &`  
#### 4) Configurer Spyder en suivant ces instructions [Lien configuration Spyder](https://gbourmaud.github.io/files/configuration_spyder_annotated.pdf).
### Si vous utilisez votre ordinateur personnel, il faudra installer Spyder.

# I) Préparation de la base de données étiquetées MNIST

La base de données de reconnaissance de chiffres manuscrits MNIST est disponible [ici](https://gbourmaud.github.io/files/intro_deep_learning/TP/TP_MLP/MNIST.tar.gz).

- Décompresser l'archive, par exemple dans `/tmp`.
- Inspecter les fichiers décompressés. Combien y-a-t-il de données d'entraînement ? de données de test ? Visualiser plusieurs images.

# II) La classe `torch.utils.data.Dataset`

La bibliothèque PyTorch contient des fonctionnalités permettant de générer automatiquement et efficacement (en utilisant plusieurs processus) des minibatchs sur CPU, notamment à travers les classes `torch.utils.data.Dataset` et `torch.utils.data.Dataloader`. 

Un `DataLoader` prend en entrée un `Dataset`. Ainsi il va falloir créer une classe `MNISTDataset` dédiée à la base de données MNIST qui héritera de `torch.utils.data.Dataset`.  La documentation de cette classe est consultable dans le terminal Spyder : `help(torch.utils.data.Dataset)` ou ici : https://pytorch.org/tutorials/beginner/basics/data_tutorial.html. 

- Créer un fichier `MNISTDataset.py`
- Créer une classe `MNISTDataset` qui hérite de `torch.utils.data.Dataset` (`class MNISTDataset(t.utils.data.Dataset):`)

Cette classe `MNISTDataset` doit contenir au moins : 
- la méthode `def __init__(self, ...):` qui s'exécute à la création de l'objet. Cette méthode a principalement pour objectif de charger en mémoire la liste des noms des images de la base de données. Attention, il ne faut surtout pas charger toutes les images de la base de données dans cette fonction. Certes cela rentrerait en mémoire pour MNIST car cette base de données est *petite* mais cela ne fonctionnerait pas pour une base de données plus grande. Le chargement d'une image s'effectue dans la méthode `def __getitem__(self, idx):`.
- la méthode `def __len__(self):` qui renvoie le nombre de données étiquetées de la base
- la méthode `def __getitem__(self, idx):` qui permet de charger et de renvoyer l'image numéro `idx` ainsi que son étiquette et toutes les autres informations jugées nécessaires.

Si vous rencontrez des difficultés pour implémenter ce `Dataset`, vous trouverez ci-après une version fonctionnelle.

In [ ]:
import torch as t
import PIL.Image as Image
import torchvision.transforms as T
import os


class MNISTDataset(t.utils.data.Dataset):
    def __init__(self, MNIST_dir):
        
        self.MNIST_dir = MNIST_dir
        self.num_classes = 10
        
        self.img_list = []
        self.label_list = []
        for i in range(self.num_classes):
            path_cur = os.path.join(self.MNIST_dir,'{}'.format(i))
            img_list_cur = os.listdir(path_cur)
            
            img_list_cur = [os.path.join('{}'.format(i), file) for file in img_list_cur]

            self.img_list += img_list_cur
            
            label_list_cur = [i] * len(img_list_cur)
            self.label_list += label_list_cur
            
    def __len__(self):
        return len(self.label_list)

    def __getitem__(self, idx):
        
        img_path = os.path.join(self.MNIST_dir, self.img_list[idx])
        
        I_PIL = Image.open(img_path)
        
        I = T.ToTensor()(I_PIL)

        return I, t.tensor(self.label_list[idx]), img_path

Dans un nouveau script `main.py`, tester ce `Dataset` en affichant 4 éléments de la base de données MNIST.

In [ ]:
import torch as t
from MNISTDataset import MNISTDataset
import torchvision.transforms as T
import matplotlib.pyplot as plt

path_MNIST_train = '/tmp/MNIST/Training'                
training_set = MNISTDataset(path_MNIST_train)

plt.figure(1)
for i in range(4):
    image, label, _ = training_set[i]
    plt.subplot(1,4,i+1)
    plt.imshow(T.ToPILImage()(image))
    plt.title('True label {}'.format(label))
    
plt.show()

# III) La classe `torch.utils.data.DataLoader`


Maintenant que le `Dataset` fonctionne, il suffit de l'utiliser lors de la création d'un `DataLoader`.

In [ ]:
batch_size = 16
train_loader = t.utils.data.DataLoader(dataset = training_set,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       num_workers=2)

Afficher les 4 premiers éléments du premier minibatch.

In [ ]:
images, labels, _ = next(iter(train_loader))

plt.figure(2)
for i in range(4):
    plt.subplot(1,4,i+1)
    plt.imshow(T.ToPILImage()(images[i,:,:,:]))
    plt.title('True label {}'.format(labels[i]))
    
plt.show()

- Quelle est la taille du tenseur `images` ? En PyTorch, un minibatch d'images est un tenseur 4D : `taille_minibatch x nombre_de_canaux x nombre_de_lignes x nombre_de_colonnes`. 
- Quelle est la taille du tenseur `labels` ?

Lors d'un entraînement avec arrêt prématuré, nous utiliserons la base de données de *testing* comme base de validation.

In [ ]:
path_MNIST_valid = '/tmp/MNIST/Testing'                
valid_set = MNISTDataset(path_MNIST_valid)
valid_loader = t.utils.data.DataLoader(dataset = valid_set,
                                       batch_size=batch_size,
                                       shuffle=False,
                                       num_workers=2)

# IV) MLP sur MNIST


Maintenant que le `DataLoader` de MNIST est prêt, vous pouvez reprendre et adapter le code du MLP obtenu à la fin du TP précédent (`TP MLP PyTorch jouet`) afin de lancer un apprentissage des paramètres du MLP sur la base de données MNIST.

---
**COMMENCER PAR LANCER UN APPRENTISSAGE SUR UN SEUL MINIBATCH POUR EVACUER RAPIDEMENT LA PLUPART DES BUGS PRESENTS DANS VOTRE CODE**

---

Vous trouverez ci-après un exemple de code fonctionnel permettant de réaliser un apprentissage d'un MLP sur MNIST.

In [ ]:
import torch
from MNISTDataset import MNISTDataset
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn as nn

torch.random.manual_seed(0)
path_MNIST_train = '/tmp/MNIST/Training'                
training_set = MNISTDataset(path_MNIST_train)

plt.figure(1)
for i in range(4):
    image, label, _ = training_set[i]
    plt.subplot(1,4,i+1)
    plt.imshow(T.ToPILImage()(image))
    plt.title('True label {}'.format(label))
    
plt.pause(1.)


batch_size = 128
train_loader = torch.utils.data.DataLoader(dataset = training_set,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       num_workers=2)
images, labels, _ = next(iter(train_loader))

plt.figure(2)
for i in range(4):
    plt.subplot(1,4,i+1)
    plt.imshow(T.ToPILImage()(images[i,:,:,:]))
    plt.title('True label {}'.format(labels[i]))
    
plt.pause(1.)

path_MNIST_valid = '/tmp/MNIST/Testing'                
valid_set = MNISTDataset(path_MNIST_valid)
valid_loader = torch.utils.data.DataLoader(dataset = valid_set,
                                       batch_size=batch_size,
                                       shuffle=False,
                                       num_workers=2)

   
        
class MLP(nn.Module):
    def __init__(self, H, input_size):
        super(MLP, self).__init__()
        
        self.C = 10
        self.D = input_size
        self.H = H
        
        
        self.fc1 = nn.Linear(self.D, self.H) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(self.H, self.C)  
        
        
    def forward(self,X):
    
        X1 = self.fc1(X) #NxH
        X2 = self.relu(X1) #NxH
        O = self.fc2(X2) #NxC
    
        return O
    

def validation(valid_loader, model):
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels, _ in valid_loader:
            images_vec = images.view(-1, 28*28)
            
            outputs = model(images_vec)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (correct, total)

#%% HYPERPARAMETERS
H = 30
lr = 1e-2 #learning rate
beta = 0.9 #momentum parameter
n_epoch = 100 #number of iterations
input_size = 784

model = MLP(H,input_size)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=beta)  
criterion = nn.CrossEntropyLoss()

num_batch = len(train_loader) #600 batches each containing 100 images = 60000 images

training_loss_v = []
valid_acc_v = []

(correct, total) = validation(valid_loader, model)
print ('Epoch [{}/{}], Valid Acc: {} %'
           .format(0, n_epoch, 100 * correct / total))
valid_acc_v.append(correct / total)

for epoch in range(n_epoch):
    
    loss_tot = 0

    for i, (images, labels,_) in enumerate(train_loader):

        # Reshape images to (batch_size, input_size), actual shape is (batch_size, 1, 28, 28)
        images_vec = images.view(-1, input_size)
            
        #Forward Pass
        O = model.forward(images_vec)
        
        #Compute Loss
        l = criterion(O, labels)
        
        #Print Loss
        loss_tot += l.item()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Batch Loss: {:.4f}' 
                   .format(epoch+1, n_epoch, i+1, num_batch, l.item()/len(labels)))
    
        #Backward Pass (Compute Gradient)
        optimizer.zero_grad()
        l.backward()
        
        #Update Parameters
        optimizer.step()    
        
    
    (correct, total) = validation(valid_loader, model)
    print ('Epoch [{}/{}], Training Loss: {:.4f}, Valid Acc: {} %'
           .format(epoch+1, n_epoch, loss_tot/len(training_set), 100 * correct / total))
    training_loss_v.append(loss_tot/len(training_set))
    valid_acc_v.append(correct / total)
    
    
#%% plot results
plt.figure(2)
plt.clf()
plt.plot(training_loss_v,'r',label='Training loss')
plt.legend()

plt.figure(3)
plt.clf()
plt.plot(valid_acc_v,'g',label='Validation accuracy')
plt.legend()

(images, labels,_) = iter(valid_loader).next()
images_vec = images.reshape(-1, 28*28)
outputs = model(images_vec)
_, predicted = torch.max(outputs.data, 1)
plt.figure(4)
plt.clf()
for i in range(7):
    for j in range(3):
        image = images[i+(7*j),:]
        plt.subplot(3,7,1+i+(7*j))
        plt.imshow(T.ToPILImage()(image))
        plt.title('True {} / Pred {}'.format(labels[i+(7*j)], predicted[i+(7*j)]))

**Influence de la plage des valeurs des données en entrée du réseau**

La fonction `ToTensor` utilisée dans le `Dataset` a automatiquement normalisé le tenseur avec des valeurs dans $[0, 1]$ alors que l'image initiale avait des valeurs dans l'intervalle $[0, 255]$. 

Lancer un entraînement en multipliant le tenseur en entrée du réseau par 255 (afin que ses valeurs soient dans l'intervalle $[0, 255]$). Vous devriez constater que l'entraînement est beaucoup plus long, ou bien qu'il n'y a carrément pas de convergence. 

**Affichages**

Le code ci-dessus réalise peu d'affichages permettant de contrôler le bon déroulement de l'apprentissage. Pour ce faire, nous allons utiliser un outil supplémentaire : **TensorBoard**.

# V) TensorBoard


TensorBoard est un outil de visualisation dédié aux expériences d'appentissage automatique. Il permet notamment de tracer des courbes au cours de l'entraînement ou encore de superposer des courbes issues de différents entraînement.
Un exemple d'utilisation est présenté ici : https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html

L'objectif de cette partie est d'utiliser cet outil, en rajoutant quelques lignes de code à votre code d'entraînement du MLP sur MNIST (essentiellement `from torch.utils.tensorboard import SummaryWriter
`, `writer = SummaryWriter()`, `writer.add_scalar(...)`)
 , pour afficher des informations relatives à l'entraînement en cours (courbe du coût d'apprentissage, courbe du coût de validation, courbe de la précision de validation, valeur du pas d'apprentissage au cours du temps, valeurs de certains gradients, etc.).

# VI) MNIST décentré


Les images de la base de données MNIST sont de taille 28x28. Dans chaque image, le chiffre se situe au centre de l'image. 

L'objectif de cette partie est de modifier le `Dataset` pour obtenir des images de taille 56x56 où le chiffre n'est plus centré. Ceci peut par exemple être implémenté dans la méthode `__get_item__` en générant une translation aléatoire (en x et en y) et en l'appliquant à l'image chargée.

Cette nouvelle base de donnée, qu'on appellera *MNISTTranslation*, correspond-elle à un problème d'apprentissage supervisé plus difficile que le problème initial ? Pourquoi ? 

Lancer un entraînement avec le MLP précédemment implémenté. Que constatez-vous ? Pourquoi ?